In [4]:
import os
import SimpleITK as sitk
import numpy as np 
import skimage
import shutil
import matplotlib.pyplot as plt

In [8]:
def conn_region(seg_path):
    seg_image = sitk.ReadImage(seg_path)
    seg_arr = sitk.GetArrayFromImage(seg_image)
    labels = skimage.measure.label(seg_arr).astype(np.uint8)    #  !!!! 这里label默认是np.int64，会导致SimpleITK-snap打不开，所以要转换为np.unit8
                                                               # 但是如果labels.dtype=np.unit8 是不可行的，会导致数组长度翻8倍，
                                                              # 具体参考 https://blog.csdn.net/lz0499/article/details/81024731
    
    return seg_image, labels
    

In [9]:
def areas_filter(labels, threshold_ratio, threshold_ereas):
    """
    labels ： 连通域生成的标签，一个连通域是一个值
    threshold： 面积阈值，将小于最大连通域的threshold倍的阈值删掉
    return labels: 删除小于阈值的标签，可含多个连通域
    """
    region_index, volumn_count  = np.unique(labels, return_counts=True)  # volumn_count : [56535334,    80007,     7763]) 
    volumn_ratio = volumn_count[1: ] / volumn_count[1: ].max() 

#     print(volumn_ratio, region_index)
    select_ratio_index = np.where(volumn_ratio < threshold_ratio)[0]    # 小于最大连通域1%的不要
    select_ereas_index = np.where(volumn_count[1: ] < threshold_ereas)[0]          # 小于1000的不要
    select_index = list(set(select_ratio_index).intersection(set(select_ereas_index)))   # 同时满足这两个条件的交集
    # print(select_ratio_index, select_ereas_index, select_index)
    select_index = np.array(select_index)
    for idx in select_index + 1:
        labels[labels == idx] = 0
    
    return labels, select_ratio_index, select_ereas_index, select_index
    

In [15]:
base_dir = '/app/liucd/CODE/zunyi/data/data_299+33/'
# base_dir = '/app/liucd/CODE/syf/data/data_dec/stage_2'
label_dir =  os.path.join(base_dir, 'DCE_labelReNew/')

threshold_ratio = 0.01
threshold_ereas = 300

for seg_name in os.listdir(label_dir):
    seg_path = os.path.join(label_dir, seg_name)
    
    seg_image, raw_labels = conn_region(seg_path)

    fileter_labels, _, _, select_index  = areas_filter(raw_labels, threshold_ratio, threshold_ereas)
    fileter_labels[fileter_labels > 0] = 1

    if len(select_index) != 0:
        print(seg_path)
        filter_img = sitk.GetImageFromArray(fileter_labels)
        filter_img.SetDirection(seg_image.GetDirection())
        filter_img.SetSpacing(seg_image.GetSpacing())
        filter_img.SetOrigin(seg_image.GetOrigin())

    #     sitk.WriteImage(filter_img, os.path.join(base_dir,'Label_Gen', 'relabel', 'DWI_relabel', seg_name))
    # else:
    #     shutil.copyfile(seg_path, os.path.join(base_dir,'Label_Gen', 'relabel', 'DWI_relabel', seg_name))


/app/liucd/CODE/zunyi/data/data_299+33/DCE_labelReNew/2022_02_26_1671393_+C.nii.gz
/app/liucd/CODE/zunyi/data/data_299+33/DCE_labelReNew/2020_06_18_1616764_+C.nii.gz
/app/liucd/CODE/zunyi/data/data_299+33/DCE_labelReNew/2019_12_04_1574144_+C.nii.gz
/app/liucd/CODE/zunyi/data/data_299+33/DCE_labelReNew/2021_09_30_1747581_+C.nii.gz
/app/liucd/CODE/zunyi/data/data_299+33/DCE_labelReNew/2021_11_24_1757935_+C.nii.gz
/app/liucd/CODE/zunyi/data/data_299+33/DCE_labelReNew/2019_05_17_1519664_+C.nii.gz
/app/liucd/CODE/zunyi/data/data_299+33/DCE_labelReNew/2017_11_02_1365106_+C.nii.gz
/app/liucd/CODE/zunyi/data/data_299+33/DCE_labelReNew/2021_12_02_1321037_+C.nii.gz
/app/liucd/CODE/zunyi/data/data_299+33/DCE_labelReNew/2019_06_19_1527959_+C.nii.gz
